In [ ]:
! pip install transformers
! pip install datasets
! pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 4.0 MB/s 
     |████████████████████████████████| 6.6 MB 46.2 MB/s 
     |████████████████████████████████| 120 kB 73.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.0 MB/s 
     |████████████████████████████████| 115 kB 66.2 MB/s 
     |████████████████████████████████| 212 kB 64.0 MB/s 
     |████████████████████████████████| 127 kB 69.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import pandas as pd

df_foodcom_source = pd.read_csv("drive/My Drive/0_Dataset_Recipes/RAW_recipes.csv", sep = ",")


In [ ]:
df_foodcom = df_foodcom_source.drop(['id', 'name','minutes', 'contributor_id', 'submitted', 'tags', 'nutrition', 'n_steps', 'description', 'ingredients', 'n_ingredients'], axis = 1)
df_foodcom.head()

,steps
0,"['make a choice and proceed with recipe', 'dep..."
1,"['preheat oven to 425 degrees f', 'press dough..."
2,"['brown ground beef in large pot', 'add choppe..."
3,['place potatoes in a large pot of lightly sal...
4,['mix all ingredients& boil for 2 1 / 2 hours ...


In [ ]:
df_foodcom.describe()

,steps
count,231637
unique,231074
top,['blend all ingredients until smooth']
freq,20


In [ ]:
# Extraire une partie des données pour créer des échantillons d'entrainement, de validation et de test, avec nombre de lignes réduits
# initialement pour économie de temps d'entrainement
nb_train = 2000
nb_test = 200

df_foodcom_train = df_foodcom.iloc[: nb_train, :]
print(df_foodcom_train.shape)

df_foodcom_validation = df_foodcom.iloc[nb_train+1:nb_train+nb_test+1, :]
print(df_foodcom_validation.shape)

df_foodcom_test = df_foodcom.iloc[nb_train+nb_test+2: nb_train+2*nb_test+2, :]
print(df_foodcom_test.shape)

(2000, 1)
(200, 1)
(200, 1)


In [ ]:
# Charger les données du format dataframe dans format dataset de HuggingFace
from datasets import load_dataset, Dataset
ds_foodcom_train = Dataset.from_pandas(df_foodcom_train)
ds_foodcom_valid = Dataset.from_pandas(df_foodcom_validation)
ds_foodcom_test = Dataset.from_pandas(df_foodcom_test)

In [ ]:
# Importer le tokeniser associé au modèle distilgpt2
model_checkpoint1 = "distilgpt2"

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint1)

# définir fonction TOKENIZE  qui standardise le texte (enlèvement de signes spéciaux & appliquer fonction de tokenisation )
def tokenize_function(row):
    text_nosign = re.sub(r'[\'\,\]\[\-\&\"\/\!\^\#]', ' ', row['steps']) 
    text_final  = re.sub(r'[\s]+', ' ', text_nosign) 
    return tokenizer(text_final)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# définir une fonction GROUP TEXTE qui concatener tous les recettes dans un seul texte, puis le découper en séquence de 50 tokens 
# BUT: avoir les séquences d'input de longeur fixe

block_size = 50

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    # Split by chunks of block size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
         }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# Combiner la fonction TOKENIZER & fonction GROUPE TEXTE dans une seule fonctionpour appliquer sur  train, validation & test datasets

def preprocess_ds (ds, batchSize):
    ds_tk = ds.map(tokenize_function, remove_columns=["steps"])
    ds_fin = ds_tk.map(
                                group_texts,
                                batched=True,
                                batch_size=batchSize,
                                num_proc=4,)
    return ds_fin

In [ ]:
ds_foodcom_train_fin = preprocess_ds(ds_foodcom_train, batchSize = 500)
ds_foodcom_valid_fin = preprocess_ds(ds_foodcom_valid, batchSize = 200)
ds_foodcom_test_fin = preprocess_ds(ds_foodcom_test, batchSize = 200)

  0%|          | 0/2000 [00:00<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1087 > 1024). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/200 [00:00<?, ?ex/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:

ds_foodcom_valid_fin


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 485
})

TRANSFORMER TRAINING

In [ ]:
from transformers import TFAutoModelForCausalLM

model = TFAutoModelForCausalLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/328M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
from transformers import create_optimizer, AdamWeightDecay
optimizer = AdamWeightDecay(lr=2e-5, weight_decay_rate=0.01)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
import tensorflow as tf
model.compile(optimizer=optimizer, jit_compile=True)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
train_set = model.prepare_tf_dataset(
    ds_foodcom_train_fin,
    shuffle=True,
    batch_size=16,
)

In [ ]:
validation_set = model.prepare_tf_dataset(
    ds_foodcom_valid_fin,
    shuffle=True,
    batch_size=16,
)

In [ ]:

model.fit(train_set, validation_data=validation_set, epochs=1)

255/255 [==============================] - 61s 142ms/step - loss: 4.1262 - val_loss: 3.8555


In [ ]:
test_sentence = "cook"

tokenized = tokenizer(test_sentence, return_tensors="np")

outputs = model.generate(**tokenized, max_length=30)

tokenizer.decode(outputs[0])

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'cook the chicken and cook for about 10 minutes or until chicken is tender and tender add the chicken and cook for about 10 minutes or until chicken is tender'

In [ ]:
model.save_pretrained("drive/My Drive/DataScientest - NLP_Mail/GNG_huggingFace")